In [1]:
import gym
import numpy as np
rm='Pong-v4'

In [2]:
%matplotlib inline
from matplotlib import animation
import matplotlib.pyplot as plt
from IPython.display import display, HTML

def display_frames_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 144)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    plt.close(anim._fig)
    display(HTML(anim.to_jshtml()))

In [3]:
from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make(rm)
# model initialization
H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
#learning_rate = 1e-3
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  I=np.asarray(I)
  I = I[35:195] # crop
  
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):
  # preprocess the observation, set input to network to be difference image
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

    # if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
    #   print('ep {}: game finished, reward: {}'.format(episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\gym\core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\gym\wrappers\step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default 

In [4]:
env.action_space

Discrete(6)

In [5]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
%time hist1 = train_model(env, model, total_episodes=6000)

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\gym\utils\passive_env_checker.py:228: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  "Core environment is written in old step API which returns one bool instead of two. "


episode 1.000000, reward total was -21.000000. running mean: -21.000000
episode 2.000000, reward total was -21.000000. running mean: -21.000000
episode 3.000000, reward total was -20.000000. running mean: -20.990000
episode 4.000000, reward total was -20.000000. running mean: -20.980100
episode 5.000000, reward total was -21.000000. running mean: -20.980299
episode 6.000000, reward total was -20.000000. running mean: -20.970496
episode 7.000000, reward total was -21.000000. running mean: -20.970791
episode 8.000000, reward total was -19.000000. running mean: -20.951083
episode 9.000000, reward total was -21.000000. running mean: -20.951572
episode 10.000000, reward total was -20.000000. running mean: -20.942057
episode 11.000000, reward total was -21.000000. running mean: -20.942636
episode 12.000000, reward total was -20.000000. running mean: -20.933210
episode 13.000000, reward total was -19.000000. running mean: -20.913878
episode 14.000000, reward total was -18.000000. running mean

episode 114.000000, reward total was -21.000000. running mean: -20.561682
episode 115.000000, reward total was -21.000000. running mean: -20.566065
episode 116.000000, reward total was -20.000000. running mean: -20.560404
episode 117.000000, reward total was -21.000000. running mean: -20.564800
episode 118.000000, reward total was -21.000000. running mean: -20.569152
episode 119.000000, reward total was -20.000000. running mean: -20.563461
episode 120.000000, reward total was -21.000000. running mean: -20.567826
episode 121.000000, reward total was -21.000000. running mean: -20.572148
episode 122.000000, reward total was -21.000000. running mean: -20.576426
episode 123.000000, reward total was -20.000000. running mean: -20.570662
episode 124.000000, reward total was -19.000000. running mean: -20.554956
episode 125.000000, reward total was -21.000000. running mean: -20.559406
episode 126.000000, reward total was -20.000000. running mean: -20.553812
episode 127.000000, reward total was -

episode 225.000000, reward total was -20.000000. running mean: -20.430588
episode 226.000000, reward total was -17.000000. running mean: -20.396282
episode 227.000000, reward total was -21.000000. running mean: -20.402319
episode 228.000000, reward total was -20.000000. running mean: -20.398296
episode 229.000000, reward total was -21.000000. running mean: -20.404313
episode 230.000000, reward total was -21.000000. running mean: -20.410270
episode 231.000000, reward total was -20.000000. running mean: -20.406167
episode 232.000000, reward total was -20.000000. running mean: -20.402105
episode 233.000000, reward total was -21.000000. running mean: -20.408084
episode 234.000000, reward total was -21.000000. running mean: -20.414004
episode 235.000000, reward total was -21.000000. running mean: -20.419864
episode 236.000000, reward total was -20.000000. running mean: -20.415665
episode 237.000000, reward total was -19.000000. running mean: -20.401508
episode 238.000000, reward total was -

episode 336.000000, reward total was -21.000000. running mean: -20.297233
episode 337.000000, reward total was -20.000000. running mean: -20.294261
episode 338.000000, reward total was -19.000000. running mean: -20.281318
episode 339.000000, reward total was -21.000000. running mean: -20.288505
episode 340.000000, reward total was -20.000000. running mean: -20.285620
episode 341.000000, reward total was -21.000000. running mean: -20.292764
episode 342.000000, reward total was -21.000000. running mean: -20.299836
episode 343.000000, reward total was -19.000000. running mean: -20.286838
episode 344.000000, reward total was -21.000000. running mean: -20.293970
episode 345.000000, reward total was -19.000000. running mean: -20.281030
episode 346.000000, reward total was -21.000000. running mean: -20.288220
episode 347.000000, reward total was -20.000000. running mean: -20.285337
episode 348.000000, reward total was -21.000000. running mean: -20.292484
episode 349.000000, reward total was -

...

episode 5512.000000, reward total was -21.000000. running mean: -19.463624
episode 5513.000000, reward total was -19.000000. running mean: -19.458987
episode 5514.000000, reward total was -19.000000. running mean: -19.454398
episode 5515.000000, reward total was -21.000000. running mean: -19.469854
episode 5516.000000, reward total was -21.000000. running mean: -19.485155
episode 5517.000000, reward total was -20.000000. running mean: -19.490304
episode 5518.000000, reward total was -20.000000. running mean: -19.495400
episode 5519.000000, reward total was -19.000000. running mean: -19.490446
episode 5520.000000, reward total was -20.000000. running mean: -19.495542
episode 5521.000000, reward total was -21.000000. running mean: -19.510587
episode 5522.000000, reward total was -18.000000. running mean: -19.495481
episode 5523.000000, reward total was -21.000000. running mean: -19.510526
episode 5524.000000, reward total was -20.000000. running mean: -19.515421
episode 5525.000000, rewa

episode 5622.000000, reward total was -19.000000. running mean: -19.374341
episode 5623.000000, reward total was -19.000000. running mean: -19.370597
episode 5624.000000, reward total was -18.000000. running mean: -19.356892
episode 5625.000000, reward total was -20.000000. running mean: -19.363323
episode 5626.000000, reward total was -20.000000. running mean: -19.369689
episode 5627.000000, reward total was -20.000000. running mean: -19.375992
episode 5628.000000, reward total was -21.000000. running mean: -19.392233
episode 5629.000000, reward total was -19.000000. running mean: -19.388310
episode 5630.000000, reward total was -19.000000. running mean: -19.384427
episode 5631.000000, reward total was -19.000000. running mean: -19.380583
episode 5632.000000, reward total was -19.000000. running mean: -19.376777
episode 5633.000000, reward total was -21.000000. running mean: -19.393009
episode 5634.000000, reward total was -20.000000. running mean: -19.399079
episode 5635.000000, rewa

episode 5732.000000, reward total was -19.000000. running mean: -19.208765
episode 5733.000000, reward total was -20.000000. running mean: -19.216677
episode 5734.000000, reward total was -20.000000. running mean: -19.224511
episode 5735.000000, reward total was -20.000000. running mean: -19.232266
episode 5736.000000, reward total was -18.000000. running mean: -19.219943
episode 5737.000000, reward total was -19.000000. running mean: -19.217743
episode 5738.000000, reward total was -18.000000. running mean: -19.205566
episode 5739.000000, reward total was -19.000000. running mean: -19.203510
episode 5740.000000, reward total was -20.000000. running mean: -19.211475
episode 5741.000000, reward total was -21.000000. running mean: -19.229361
episode 5742.000000, reward total was -18.000000. running mean: -19.217067
episode 5743.000000, reward total was -21.000000. running mean: -19.234896
episode 5744.000000, reward total was -19.000000. running mean: -19.232547
episode 5745.000000, rewa

episode 5842.000000, reward total was -19.000000. running mean: -19.287939
episode 5843.000000, reward total was -17.000000. running mean: -19.265059
episode 5844.000000, reward total was -21.000000. running mean: -19.282409
episode 5845.000000, reward total was -20.000000. running mean: -19.289585
episode 5846.000000, reward total was -20.000000. running mean: -19.296689
episode 5847.000000, reward total was -21.000000. running mean: -19.313722
episode 5848.000000, reward total was -16.000000. running mean: -19.280585
episode 5849.000000, reward total was -18.000000. running mean: -19.267779
episode 5850.000000, reward total was -20.000000. running mean: -19.275101
episode 5851.000000, reward total was -19.000000. running mean: -19.272350
episode 5852.000000, reward total was -18.000000. running mean: -19.259627
episode 5853.000000, reward total was -18.000000. running mean: -19.247030
episode 5854.000000, reward total was -20.000000. running mean: -19.254560
episode 5855.000000, rewa

episode 5952.000000, reward total was -21.000000. running mean: -19.344722
episode 5953.000000, reward total was -20.000000. running mean: -19.351275
episode 5954.000000, reward total was -20.000000. running mean: -19.357762
episode 5955.000000, reward total was -17.000000. running mean: -19.334185
episode 5956.000000, reward total was -21.000000. running mean: -19.350843
episode 5957.000000, reward total was -20.000000. running mean: -19.357335
episode 5958.000000, reward total was -21.000000. running mean: -19.373761
episode 5959.000000, reward total was -20.000000. running mean: -19.380024
episode 5960.000000, reward total was -17.000000. running mean: -19.356223
episode 5961.000000, reward total was -21.000000. running mean: -19.372661
episode 5962.000000, reward total was -19.000000. running mean: -19.368935
episode 5963.000000, reward total was -19.000000. running mean: -19.365245
episode 5964.000000, reward total was -19.000000. running mean: -19.361593
episode 5965.000000, rewa